# Algoritmo 1 - Triangle Splitting
Começando com o graham scan para achar o hull

In [34]:
import os
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import random
from dataclasses import dataclass
from collections import defaultdict
import math
import imageio

@dataclass
class Point:
    x: float
    y: float

# Criando classe fodder para o caso de não querer animação
class NullAnimator:
    def __init__(self, *args, **kwargs): pass
    def save_frame(self, _): pass
    def animate(self, _): pass
    hull = []

def polar_angle(p0, p1):
    return math.atan2(p1.y - p0.y, p1.x - p0.x)

def distance(p0, p1):
    return (p1.x - p0.x)**2 + (p1.y - p0.y)**2

def orientation_cross(p, q, r):
    """Retorna positivo se p→q→r faz curva à esquerda, 0 se colinear, negativo se direita."""
    return (q.x - p.x)*(r.y - p.y) - (q.y - p.y)*(r.x - p.x)

def graham_scan(pts, animator=NullAnimator()):
    points = [Point(*pt) if not isinstance(pt, Point) else pt for pt in pts]
    if len(points) < 3:
        animator.hull = points
        animator.save_frame(None)
        return points

    # 1. Ponto mais à esquerda
    p0 = min(points, key=lambda p: (p.x, p.y))

    # 2. Agrupar pontos por ângulo polar e pegar apenas o mais distante em cada grupo
    angle_map = defaultdict(list)
    for p in points:
        if p == p0:
            continue
        angle = polar_angle(p0, p)
        angle_map[angle].append(p)

    filtered_pts = []
    for angle in sorted(angle_map.keys()):
        farthest = max(angle_map[angle], key=lambda p: distance(p0, p))
        filtered_pts.append(farthest)

    sorted_pts = [p0] + filtered_pts

    # 3. Inicializa pilha
    stack = [sorted_pts[0], sorted_pts[1]]
    animator.hull = list(stack)
    animator.save_frame(sorted_pts[1])

    # 4. Processar os demais pontos
    for p in sorted_pts[2:]:
        while len(stack) >= 2 and orientation_cross(stack[-2], stack[-1], p) <= 0:
            stack.pop()
            animator.hull = list(stack)
            animator.save_frame(p)

        stack.append(p)
        animator.hull = list(stack)
        animator.save_frame(p)

    # 5. Hull final
    animator.hull = stack
    animator.save_frame(None)
    return stack

Voltei para a animação em funções, pois agora há mais etapas

In [39]:
def plot_step(points, lines, filename, title=""):
    """Gera o plot de cada etapa"""
    plt.figure()
    x, y = zip(*points)
    plt.scatter(x, y, c='black')

    for line in lines:
        xs, ys = zip(*line)
        plt.plot(xs, ys, 'b-')

    plt.title(title)
    plt.axis('equal')
    plt.savefig(filename)
    plt.close()

def trivial_triangulation(hull):
    """Triangulação inicial. Traça uma aresta entre o ponto mais à esquerda do Hull e cada outro ponto não-adjacente do Hull"""
    lines = []
    leftmost = min(hull, key=lambda p: p[0])
    left_idx = hull.index(leftmost)

    for i, p in enumerate(hull):
        if i not in [(left_idx - 1) % len(hull), left_idx, (left_idx + 1) % len(hull)]:
            lines.append((leftmost, p))
    return lines

def point_in_triangle(p, a, b, c):
    """Verifica se ponto está em triângulo"""
    def sign(p1, p2, p3):
        return (p1[0] - p3[0]) * (p2[1] - p3[1]) - \
               (p2[0] - p3[0]) * (p1[1] - p3[1])
    
    b1 = sign(p, a, b) < 0.0
    b2 = sign(p, b, c) < 0.0
    b3 = sign(p, c, a) < 0.0

    return (b1 == b2) and (b2 == b3)

def triangle_split(points, hull, folder=None):
    """Insere os pontos no triângulo"""
    triangles = []
    leftmost = min(hull, key=lambda p: p[0])
    left_idx = hull.index(leftmost)

    # Triangulação inicial
    triangles = []
    n = len(hull)
    for i in range(n):
        if i != left_idx and (i + 1) % n != left_idx:
            a = leftmost
            b = hull[i]
            c = hull[(i + 1) % n]
            triangles.append((a, b, c))

    inner_points = [p for p in points if p not in hull]
    step = 2

    for p in inner_points:
        # Tenta encontrar o triângulo que contém p
        for i, (a, b, c) in enumerate(triangles):
            if point_in_triangle(p, a, b, c):
                # Remove o triângulo antigo
                old = triangles.pop(i)
                # Adiciona os 3 novos triângulos
                triangles.extend([(p, a, b), (p, b, c), (p, c, a)])
                break  # Só insere em um triângulo

        if folder:
            lines = []
            for t in triangles:
                lines.extend([(t[0], t[1]), (t[1], t[2]), (t[2], t[0])])
            plot_step(points, lines, f"{folder}/step_{step:03d}.png",
                      title=f"Inserção do ponto {p}")
            step += 1

def triangle_split_animation(points, hull_function, output_video=None, animate=False, test_id=None, final_plot=True):
    """Pipeline principal"""
    if test_id is None:
        test_id = len(points)
    
    if output_video is None:
        output_video = f"simulacoes_triangle_splitting/{test_id}/triangle_split.mp4"

    folder = f"simulacoes_triangle_splitting/{test_id}" if animate else None
    if folder:
        os.makedirs(folder, exist_ok=True)

    hull = [(p.x, p.y) for p in hull_function(points)]
    if animate:
        # Etapa 0: Convex Hull
        plot_step(points, [(hull[i], hull[(i+1)%len(hull)]) for i in range(len(hull))],
                  f"{folder}/step_000.png", "Convex Hull")

        # Etapa 1: Triangulação Trivial
        leftmost = min(hull, key=lambda p: p[0])
        idx = hull.index(leftmost)
        tri_lines = []
        for i in range(len(hull)):
            if i != idx and (i + 1) % len(hull) != idx:
                tri_lines.append((leftmost, hull[i]))
                tri_lines.append((leftmost, hull[(i + 1) % len(hull)]))
        all_lines = [(hull[i], hull[(i+1)%len(hull)]) for i in range(len(hull))] + tri_lines
        plot_step(points, all_lines, f"{folder}/step_001.png", "Triangulação trivial")

    # Etapa 2 em diante: inserção dos pontos internos
    triangle_split(points, hull, folder=folder)

    # Mostrar triangulação final na tela
    final_triangles = []
    leftmost = min(hull, key=lambda p: p[0])
    idx = hull.index(leftmost)
    n = len(hull)

    # Triangulação trivial
    for i in range(n):
        if i != idx and (i + 1) % n != idx:
            a = leftmost
            b = hull[i]
            c = hull[(i + 1) % n]
            final_triangles.append((a, b, c))

    inner_points = [p for p in points if p not in hull]
    for p in inner_points:
        for i, (a, b, c) in enumerate(final_triangles):
            if point_in_triangle(p, a, b, c):
                final_triangles.pop(i)
                final_triangles.extend([(p, a, b), (p, b, c), (p, c, a)])
                break

    print("Triângulos finais:")
    for t in final_triangles:
        print(f"{t[0]} - {t[1]} - {t[2]}")

    if final_plot:
        # Plot final na tela
        plt.figure()
        for a, b, c in final_triangles:
            plt.plot([a[0], b[0]], [a[1], b[1]], 'r-')
            plt.plot([b[0], c[0]], [b[1], c[1]], 'r-')
            plt.plot([c[0], a[0]], [c[1], a[1]], 'r-')
        x, y = zip(*points)
        plt.scatter(x, y, c='black')
        plt.title("Triangulação Final")
        plt.axis('equal')
        plt.show()
    
    if animate:
        images = []
        for filename in sorted(os.listdir(folder)):
            if filename.endswith(".png"):
                images.append(imageio.imread(os.path.join(folder, filename)))
        imageio.mimsave(output_video, images, fps=2)


# Para testar o algoritmo, mude a váriavel "pts" abaixo e rode o algoritmo. 
Se quiser uma simulação visual com o matplotlib, mude a variável "animated" para "True". Se quiser gerar pontos aleatórios, utilize a função "generate_random_points" como pode ser visto abaixo. Caso contrário, passe uma array de pontos no formato [(a,b), (c,d), (e,f), ... (y,z)]. Se quiser ver apenas um plot da triangulação no final mude final_plot para True

In [38]:
random.seed(42)
def generate_random_points(num_points):
    return [(random.uniform(-20, 20), random.uniform(-20,20)) for _ in range(num_points)]

In [59]:
# MUDE APENAS ESSAS VARIÁVEIS
pts = generate_random_points(30)
animate=False
final_plot=False


if __name__ == "__main__":
    random.seed(42)
    print(f"Pontos gerados: {pts}")
    triangle_split_animation(pts, graham_scan, animate=animate, final_plot=final_plot)
    

Pontos gerados: [(19.901504259804412, 0.38105174705858147), (-16.363623513048246, -18.115344983010615), (-15.614034785973633, 5.0978416681236), (11.683174574518564, -3.1136013280126384), (-17.458891753921716, -4.735228539738529), (19.84485520960387, 1.1645738039654816), (18.84313510454473, 14.431188089379923), (-19.540759122287213, 8.828872774407785), (7.268414761062992, 1.4788132163518064), (-9.326992401898288, 5.638471943192322), (-15.537913056164943, -2.6093899732357997), (-1.8510517468317431, 18.152637100843208), (15.034117615127762, -9.46443796995637), (0.023444522011931923, -12.853924778794745), (16.50511357379282, 14.820742793470679), (-8.062208342054685, 5.557979794640207), (4.358808457526891, -13.886429258014608), (10.50043200300605, 1.5751612047850294), (11.14505914522233, 1.2141468878071002), (-19.97712415488226, -7.033757719813076), (-19.220930304566707, 17.163944650584682), (15.14887511292737, 13.266621174447181), (-7.699434983893543, -17.6829933402325), (15.12038396816161

# Algoritmo 2 - Incremental

Apenas copiei o algoritmo do Hull incremental e modifiquei algumas das funções

In [54]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import os
import random
from dataclasses import dataclass
from collections import defaultdict
import math

EPSILON = 1e-9
ROT_ANGLE = 1e-4
PERTURB   = 1e-7

def orient(a,b,c):
    return (b.x-a.x)*(c.y-a.y) - (b.y-a.y)*(c.x-a.x)

class Point:
    def __init__(self,x,y): self.x, self.y = x,y
    def rotate(self,a):
        c,s = math.cos(a), math.sin(a)
        return Point(self.x*c-self.y*s, self.x*s+self.y*c)
    def perturbed(self):
        return Point(self.x+random.uniform(-PERTURB,PERTURB),
                     self.y+random.uniform(-PERTURB,PERTURB))
    def __repr__(self): return f"({self.x:.4f},{self.y:.4f})"
    
    def __iter__(self):  
        yield self.x
        yield self.y

class IncrementalConvexHull:
    def __init__(self, points, animated=False):
        self.points   = [Point(*p) if not isinstance(p,Point) else p for p in points]
        self.hull     = []
        self.tri      = []         
        self.frames   = []
        self.animated = animated

    def preprocess(self):
        if len({p.x for p in self.points}) != len(self.points):
            self.points = [p.rotate(ROT_ANGLE) for p in self.points]
            rotated = True
        else: rotated = False
        self.points.sort(key=lambda p:(p.x,p.y))
        if rotated: self.points = [p.rotate(-ROT_ANGLE) for p in self.points]

    def init_hull(self):
        """Garantindo tratamento de pontos colineares para o algoritmo de triangulação"""
        col = [self.points[0]]
        for pt in self.points[1:]:
            if len(col) == 1:             
                col.append(pt)
                continue
            if abs(orient(col[-2], col[-1], pt)) < EPSILON:
                col.append(pt)            
            else:
                break                     

        if len(col) == len(self.points):
            raise ValueError("Todos os pontos são colineares — "
                            "não há triangulação 2-D possível.")

        p_nc = self.points[len(col)]      

        self.tri = []                      
        for i in range(len(col)-1):        
            a, b = col[i], col[i+1]
            self.tri.append((p_nc, a, b))

        left, right = col[0], col[-1]
        if orient(left, right, p_nc) > 0:   
            right, p_nc = p_nc, right
        self.hull = [left, right, p_nc]

        if self.animated:
            self.save_frame(None)
        return len(col)


    def is_inside(self,p):
        for a,b in zip(self.hull, self.hull[1:]+self.hull[:1]):
            if orient(a,b,p) < -EPSILON: return False
        return True


    def add_point(self,p):
        """Adicionando o ponto no hull além da triangulação para facilitar o cálculo da tangente"""
        if self.is_inside(p):                        
            if self.animated: self.save_frame(p)
            return

        n = len(self.hull)
        visible = [i for i in range(n)
                   if orient(self.hull[i], self.hull[(i+1)%n], p) > EPSILON]


        for i in visible:                             
            a, b = self.hull[i], self.hull[(i+1)%n]    
            self.tri.append((a, b, p))                 

        if len(visible) > 1:            
            for idx in visible[::-1][:-1]:
                del self.hull[idx]

        self.hull.insert(visible[0]+1, p)            

        if self.animated: self.save_frame(p)


    def compute(self):
        self.preprocess()
        index = self.init_hull()
        for p in self.points[index+1:]: self.add_point(p)
        return self.hull, self.tri                 


    def save_frame(self,current):
        self.frames.append({"hull":list(self.hull),
                            "tri": list(self.tri),     
                            "pts": list(self.points),
                            "cur": current})

    def animate(self, test_id):
        fig,ax = plt.subplots(figsize=(6,6))
        def upd(i):
            if i >= len(self.frames)-3: i = len(self.frames)-1
            f = self.frames[i]; ax.clear()
            ax.set_title(f"Etapa {i}")
            ax.set_xlim(min(p.x for p in f["pts"])-1, max(p.x for p in f["pts"])+1)
            ax.set_ylim(min(p.y for p in f["pts"])-1, max(p.y for p in f["pts"])+1)
            ax.scatter([p.x for p in f["pts"]], [p.y for p in f["pts"]], color='blue')
            if f["cur"]: ax.scatter(f["cur"].x, f["cur"].y, color='green', s=90)

            # desenha todos os triângulos já formados
            for t in f["tri"]:
                tx,ty = zip(*(t+(t[0],)))  # fecha ciclo
                ax.plot(tx,ty, alpha=.4, lw=2, color='red')

            ax.grid(True)
        FuncAnimation(fig, upd, frames=len(self.frames)+2,
                      interval=800, repeat=False)\
            .save(f"simulacoes_incremental/triangulação_incremental_{test_id}.mp4",
                  writer="ffmpeg", fps=1)
        plt.close(fig)

    def print_triangulation(self):             
        for k,(a,b,c) in enumerate(self.tri,1):
            print(f"{k:02}: {a} - {b} - {c}")


# Para testar o algoritmo, mude a váriavel "pts" abaixo e rode o algoritmo. 
Se quiser uma simulação visual com o matplotlib, mude a variável "animated" para "True". Se quiser gerar pontos aleatórios, utilize a função "generate_random_points" como pode ser visto abaixo. Caso contrário, passe uma array de pontos no formato [(a,b), (c,d), (e,f), ... (y,z)].

In [45]:
random.seed(42)
def generate_random_points(num_points):
    return [(random.uniform(-20, 20), random.uniform(-20,20)) for _ in range(num_points)]

In [58]:
# ESSAS SÃO AS VARIÁVEIS QUE VOCÊ DEVE MUDAR
pts = generate_random_points(20)
animated = False


os.makedirs("simulacoes_incremental/", exist_ok=True)
print(f"Gerando Triangulação para os {len(pts)} pontos: {pts}")
triang_solver = IncrementalConvexHull(pts, animated)
triang = triang_solver.compute()
print(f"Triangulação encontrada: {triang[1]}")
if animated == True:
    triang_solver.animate(test_id=len(pts))
    print(f"Simulação salva em simulacoes_incremental/triangulacao_incremental_{len(pts)}.mp4")

Gerando Triangulação para os 20 pontos: [(-15.959942823610834, -8.881055875596315), (5.427377770576008, -5.406712841196629), (-5.192761315324695, -11.619718769140492), (-9.320887118035465, 17.46618350849976), (5.921415409863741, 4.3652402266795285), (-13.15445407207612, 9.165071918013968), (-13.463900249522865, -4.821782329694088), (19.58093402546381, 5.599990394163715), (2.2779897509858493, 7.384570039594983), (13.714076807592384, 11.039996461849793), (-10.838077121435825, -18.71599024383849), (-7.381878077636724, -9.29036496097189), (-11.56068625654694, 17.716388573402178), (15.054705058906755, -7.412884768060884), (6.2175466117952, -4.1747239575734305), (16.58190358962174, -1.6459258965040462), (-9.404793340077902, -10.134899692240662), (2.4547253665260307, -9.49033565908259), (3.3834396089416217, 15.912915344099076), (-4.023979794384109, -11.227169633708666)]
Triangulação encontrada: [((-13.1545,9.1651), (-15.9599,-8.8811), (-13.4639,-4.8218)), ((-13.1545,9.1651), (-13.4639,-4.8218